# House Prices Prediction

Based on https://www.kaggle.com/c/house-prices-advanced-regression-techniques#description competition

# Import Data and Exploratory Analysis

In [1]:
# import the packages we will need to extend the capabilities of Python
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

In [2]:
# import the two data files and convert them into Pandas Dataframes
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
# get the dimensions of the training dataframe
train.shape

(1460, 81)

In [4]:
# group the columns in the training data by type of data
train.columns.to_series().groupby(train.dtypes).groups

{dtype('int64'): Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
        'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
        'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
       dtype='object'),
 dtype('float64'): Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object'),
 dtype('O'): Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 

In [5]:
# display the first few rows of the training data
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Handling Missing Values

In [6]:
# print out a list of columns, sorted by their number of missing values
train.isna().sum().sort_values(ascending=False).head(19)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

In [7]:
# characterize the columns with missing values by their data type
train.isna().sum().sort_values(ascending=False).head(19).groupby(train.dtypes).groups

{dtype('float64'): Index(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'], dtype='object'),
 dtype('O'): Index(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageCond',
        'GarageType', 'GarageFinish', 'GarageQual', 'BsmtExposure',
        'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType',
        'Electrical'],
       dtype='object')}

In [8]:
# the missing values in certain numeric columns should be set to zero, 
# since they represent optional features of a home 
impute_to_0 = ['LotFrontage', 'GarageYrBlt', 'MasVnrArea']

for df in train,test:
    for col in impute_to_0:
        df[col].fillna(0,inplace=True)
        if col != 'SalePrice':
            new_col_name = col+"_was_missing"
            df[new_col_name] = df[col].isnull()

In [9]:
# the missing values in the other categorical columns can be replaced with the word "Missing"
optionalMissingCats = ['PoolQC', 'Alley', 'Fence', 'FireplaceQu', 'BsmtExposure', 'MiscFeature', 'GarageFinish',
        'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'Electrical','GarageCond','GarageQual',
        'GarageType', 'MasVnrType']

for df in train,test:
    for col in optionalMissingCats:
        #if df[col].isna().sum()>0:
        df[col].fillna('Missing',inplace=True)
        if col != 'SalePrice':
            new_col_name = col+"_was_missing"
            df[new_col_name] = df[col].isnull()

In [10]:
# preview the imputed values
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,GarageFinish_was_missing,BsmtFinType2_was_missing,BsmtFinType1_was_missing,BsmtCond_was_missing,BsmtQual_was_missing,Electrical_was_missing,GarageCond_was_missing,GarageQual_was_missing,GarageType_was_missing,MasVnrType_was_missing
0,1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,...,False,False,False,False,False,False,False,False,False,False
1,2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,...,False,False,False,False,False,False,False,False,False,False
2,3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,...,False,False,False,False,False,False,False,False,False,False
3,4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,...,False,False,False,False,False,False,False,False,False,False
4,5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,...,False,False,False,False,False,False,False,False,False,False


# Feature Engineering

In [11]:
# convert the numeric month values into categorical values, so each can be given its own dummy column
months_dct = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
for df in train,test:
    df['MonthSold'] = df['MoSold'].map(months_dct)
    df.drop(['MoSold'],axis=1)

In [12]:
# drop outlier values/rows from a few numeric, well-populated columns 
# in the training data that have skewed distributions
trainDroplst = ['LotArea','1stFlrSF','SalePrice']

for col in trainDroplst:
    colMean = train[col].mean()
    colStd = train[col].std()
    ThreeColStd = colStd * 3
    lowLimit = round(max(colMean - ThreeColStd,0),0)
    hiLimit = round(colMean + ThreeColStd,0)
    train = train[train[col] < hiLimit]

In [13]:
# add additional columns that capture interactions between other features
for df in train,test:
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['TotalBaths'] = df['BsmtHalfBath'] + df['FullBath'] + df['HalfBath']
    df['Basement'] = df['BsmtHalfBath'].map(lambda x: 0 if x == 0 else 1)
    df['SFperRoom'] = df['TotalSF'] / (df['TotRmsAbvGrd'] + df['Basement'])
    df['YearsUntilSold'] = df['YrSold'] - df['YearBuilt']

In [14]:
# convert the numeric year sold values into categorical values, so each can be given its own dummy column
years_dct = {2006:'2006',2007:'2007',2008:'2008',2009:'2009',2010:'2010'}
for df in train,test:
    df['YearSold'] = df['YrSold'].map(years_dct)
    df.drop(['YrSold'],axis=1)

In [15]:
# Reduce the magnitude of the columns that contain years
yearCols = ['YearBuilt','YearRemodAdd','GarageYrBlt']
for df in train, test:
    for col in yearCols:
        df[col] = df[col] - df[col].min()

In [16]:
# print the changed dimensions of the training data
train.shape

(1416, 107)

In [17]:
# confirm that the test data has 1459 rows
test.shape

(1459, 106)

## Dummy Columns for Categorical Features

In [18]:
catCols = train.select_dtypes(exclude=["number"])
categorical_features = list(catCols.columns)

In [19]:
Xtraincat= pd.get_dummies(train[categorical_features])
train = pd.concat([train, Xtraincat], axis=1)
train = train.drop(categorical_features,axis=1)
Xtestcat= pd.get_dummies(test[categorical_features])
test = pd.concat([test, Xtestcat], axis=1)
test = test.drop(categorical_features,axis=1)

In [20]:
# check that there are no categorical columns left in the training data
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1416 entries, 0 to 1459
Columns: 327 entries, Id to YearSold_2010
dtypes: float64(4), int64(39), uint8(284)
memory usage: 879.5 KB


In [21]:
# preview the expanded number of columns in the training data
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,MonthSold_Mar,MonthSold_May,MonthSold_Nov,MonthSold_Oct,MonthSold_Sep,YearSold_2006,YearSold_2007,YearSold_2008,YearSold_2009,YearSold_2010
0,1,60,65.0,8450,7,5,131,53,196.0,706,...,0,0,0,0,0,0,0,1,0,0
1,2,20,80.0,9600,6,8,104,26,0.0,978,...,0,1,0,0,0,0,1,0,0,0
2,3,60,68.0,11250,7,5,129,52,162.0,486,...,0,0,0,0,1,0,0,1,0,0
3,4,70,60.0,9550,7,5,43,20,0.0,216,...,0,0,0,0,0,1,0,0,0,0
4,5,60,84.0,14260,8,5,128,50,350.0,655,...,0,0,0,0,0,0,0,1,0,0


In [22]:
# contrast the number of columns in the training data with the slightly smaller number of columns in the test data
test.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,MonthSold_Mar,MonthSold_May,MonthSold_Nov,MonthSold_Oct,MonthSold_Sep,YearSold_2006,YearSold_2007,YearSold_2008,YearSold_2009,YearSold_2010
0,1461,20,80.0,11622,5,6,82,11,0.0,468.0,...,0,0,0,0,0,0,0,0,0,1
1,1462,20,81.0,14267,6,6,79,8,108.0,923.0,...,0,0,0,0,0,0,0,0,0,1
2,1463,60,74.0,13830,5,5,118,48,0.0,791.0,...,1,0,0,0,0,0,0,0,0,1
3,1464,60,78.0,9978,6,6,119,48,20.0,602.0,...,0,0,0,0,0,0,0,0,0,1
4,1465,120,43.0,5005,8,5,113,42,0.0,263.0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
# remove columns from the training data that are not in the test data and print dimensions
# to ensure the training data has just one more column (the target) than the test data
testCols = list(test.columns)
trainCols = list(train.columns)
trainCols.remove('SalePrice')
extraTrain = list(set(trainCols)-set(testCols))
train = train.drop(extraTrain,axis=1)
train.shape

(1416, 309)

## Evaluation

In [24]:
# split the training data into training and testing portions to validate the model that will be fit to the data
X = train.drop(["SalePrice"],axis=1)
y = train["SalePrice"]
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.25)

In [25]:
# convert the target column with a log transformation, run a regression, 
# and then convert the target column back to its original values
regr_trans = TransformedTargetRegressor(regressor=RidgeCV(),
                                        func=np.log1p,
                                        inverse_func=np.expm1)

In [26]:
# partition the training portion into four folds, and shuffle them, as part of the validation process
param_grid = {}
gscv = GridSearchCV(regr_trans, param_grid, iid=False, cv=4, return_train_score=False)

In [27]:
# fit the regression model to the training portion features and the training portion target
gscv.fit(train_X, train_y)

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=TransformedTargetRegressor(check_inverse=True, func=<ufunc 'log1p'>,
              inverse_func=<ufunc 'expm1'>,
              regressor=RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False),
              transformer=None),
       fit_params=None, iid=False, n_jobs=None, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

In [28]:
# print the parameters for the best fitting model
print(gscv.best_estimator_)

TransformedTargetRegressor(check_inverse=True, func=<ufunc 'log1p'>,
              inverse_func=<ufunc 'expm1'>,
              regressor=RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False),
              transformer=None)


In [29]:
# come up with a prediction of the target values for the test portion of the data, 
# based on the parameters for the best fitting model
ypred = gscv.best_estimator_.predict(test_X)

In [30]:
medPrice = round(train["SalePrice"].mean(),0)
mae = round(mean_absolute_error(ypred, test_y),0)
print("Mean Absolute Error is " + str(mae) + ", compared to the " + str(medPrice) + " average sale price")

Mean Absolute Error is 12510.0, compared to the 173855.0 average sale price


## Prepare Kaggle Submission

In [31]:
# find any remaining missing values in the original test data
test.isna().sum().sort_values(ascending=False).head(12)

BsmtHalfBath         2
BsmtFullBath         2
TotalBaths           2
SFperRoom            1
BsmtFinSF1           1
GarageArea           1
BsmtUnfSF            1
BsmtFinSF2           1
TotalBsmtSF          1
GarageCars           1
TotalSF              1
HouseStyle_1.5Unf    0
dtype: int64

In [32]:
# replace the missing values in the numeric columns of the original test data with zeroes
numTest = test.select_dtypes(include='number')
numTestCols = list(numTest.columns)
for col in numTestCols:
    test[col].fillna(0,inplace=True)

In [33]:
# prepare a DataFrame for the submission file
submission = pd.DataFrame(test['Id'])
submission['SalePrice'] = gscv.best_estimator_.predict(test)

In [34]:
# convert the submission dataframe into a csv file, formatted as Kaggle requires
submission.to_csv('submission.csv',index=False,index_label=False)

In [35]:
print("The submission file has been saved to the same folder as this program.")

The submission file has been saved to the same folder as this program.
